# AFLOW-CHULL

## Requirements for this Jupyter notebook:
- AFLOW binary (v3.1.225 and higher) in your path (AFLOW-install.ipynb)
- aflow_chull_plotter.py and aflow_chull.py
- Install necessary python packages: <code>pip install -r requirements.txt</code> (python 2) or <code>pip3 install -r requirements.txt</code> (python 3)

### requirements.txt
    attrs>=18.2.0
    certifi>=2018.11.29
    chardet>=3.0.4
    decorator>=4.3.2
    idna>=2.8
    ipython-genutils>=0.2.0
    jsonschema>=3.0.0
    jupyter>=1.0
    nbformat>=4.4.0
    numpy>=1.16.2
    plotly>=3.6.1
    pyrsistent>=0.14.11
    pytz>=2018.9
    requests>=2.21.0
    retrying>=1.3.3
    six>=1.12.0
    traitlets>=4.3.2
    urllib3>=1.24.1

In [ ]:
#!/usr/bin/env python

import json
import subprocess
import os
import re
import numpy as np
from random import sample
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from pprint import pprint

# Base class for AFLOW-CHULL
class CHull:

    def __init__(self, aflow_executable='aflow'):
        self.aflow_executable = aflow_executable

    def aflow_command(self, cmd):
        #print('cmd: ' + cmd)
        try:
            return subprocess.check_output(
                self.aflow_executable + cmd,
                shell=True
            )
        except subprocess.CalledProcessError:
            print('Error aflow executable not found at: ' + self.aflow_executable)

    def get_hull(self, alloy = None, options = None):
        command = ' --chull --force'
        if options:
            command += ' ' + options

        output = self.aflow_command(
            command + ' --print=json --screen_only --alloy=' + alloy
        )
        #print('output: ' + output)
        res_json = json.loads(output)
        return res_json

    def get_distance_to_hull(self, alloy = None, hull = None, off_hull_point = None, options = None):
        res_json = {}
        if alloy is not None:
            alloy=alloy.strip()
            if not alloy:
                print("ERROR: alloy input empty")
                return res_json
            command = ' --chull --distance_to_hull=' + off_hull_point
            if options:
                command += ' ' + options
            output = self.aflow_command(
                command + ' --print=json --screen_only --alloy=' + alloy
            )
            #print('output: ' + output)
            res_json = json.loads(output)
        else:
            for point in hull['points_data']:
                if point['auid'] == off_hull_point:
                    return { off_hull_point: point['enthalpy_formation_atom_difference'] }
        return res_json

    def get_stability_criterion(self, alloy = None, hull = None, hull_point = None, options = None):
        res_json = {}
        if alloy is not None:
            alloy=alloy.strip()
            if not alloy:
                print("ERROR: alloy input empty")
                return res_json
            command = ' --chull --stability_criterion=' + hull_point
            if options:
                command += ' ' + options
            output = self.aflow_command(
                command + ' --print=json --screen_only --alloy=' + alloy
            )
            #print('output: ' + output)
            res_json = json.loads(output)
        else:
            for point in hull['points_data']:
                if point['auid'] == hull_point:
                    return { hull_point: point['stability_criterion'] }
        return res_json

    def get_hull_energy(self, alloy = None, composition = None, options = None):
        command = ' --chull --hull_energy=' + ','.join([ str(comp) for comp in composition ])
        if options:
            command += ' ' + options

        output = self.aflow_command(
            command + ' --print=json --screen_only --alloy=' + alloy
        )
        #print('output: ' + output)
        res_json = json.loads(output)
        return res_json

# Base class for AFLOW-CHULL plotting
class CHullPlotter:
    def __init__(self, aflow_executable = '~/bin/aflow'):
        # Set-up notebook for offline mode
        init_notebook_mode(connected=True)

        # Load CHull in case a new json must be created
        self.chull = CHull(aflow_executable = aflow_executable)

    def plot(self, alloy = None, hull = None):
        if alloy is not None:
            alloy=alloy.strip()
            if not alloy:
                print("ERROR: alloy input empty")
                return
            hull = self.chull.get_hull(alloy)
        else:
            if hull is None:
                print("ERROR: no hull found")
                return
            alloy=hull["hull_data"]["alloy"]
        # seperate alloy into elements
        ems = re.findall('[A-Z][^A-Z]*', alloy)
        em1, em2, em3 = ems[0], ems[1], ems[2] if len(ems) == 3 else None
        # ----- Load CHull json -----
        if not em3:
            # 2D Cartesian plot
            _Two_Species_Plotter().plot_CHull(hull, em1, em2)
        else:
            # Ternary plot
            _Three_Species_Plotter().plot_CHull(hull, em1, em2, em3)

# For plotting 2 species alloys (2D cartesian)
class _Two_Species_Plotter:
    def plot_CHull(self, hull, em1, em2):
        # Set colors, randomly picked from {red, blue, green}
        color_choices = [[1,0,0], [0,1,0], [0,0,1]]
        colors = sample(color_choices, 2)
        left_color, right_color = colors[0], colors[1]

        # ----- Create scatter for edges -----
        # Grab edges
        edges = [dic['vertices_position'] for dic in list(hull['facets_data'].values())[0]]
        # Create scatter for edges
        hull_scatter = self._create_edges_scatter(edges)

        # ----- Create scatter for points -----
        points_scatter = self._create_points_scatter(hull['points_data'], em1, left_color, right_color)

        # ----- Plot -----
        fig = self._create_figure(hull_scatter, points_scatter, em1, em2)
        iplot(fig)

    def _create_edges_scatter(self, edges):
        edge_xs = []
        edge_ys = []
        for edge in edges:
            point1 = edge[0]
            edge_xs.append(1.0 - point1[0])  # Flip x values because they are reversed in the json output for some reason
            edge_ys.append(point1[1] * 1000.0)  # Multiply by 1000 to match with the points

        hull_scatter = go.Scatter(
            x=np.array(edge_xs),
            y=np.array(edge_ys),
            mode='lines',
            name='hull',
            hoverinfo='none'
        )
        return hull_scatter

    def _create_points_scatter(self, points_data, em1, left_color, right_color):
        colors, auids, urls, c_name, xs, ys = [], [], [], [], [], []

        # First grab points from the points data
        for compound in points_data:
            # Add AUID and url
            urls.append(compound['url_entry_page'])
            auids.append(compound['auid'])
            # Add compound name
            c_name.append(compound['compound'])
            # Set y: enthalpy_formation_atom
            y = compound['enthalpy_formation_atom']
            # Set x
            frac = compound['fractional_compound']
            # If 1 element then figure out if first or second element
            if compound['nspecies'] == 1:
                x = 0.0 if frac == em1 else 1.0
            # o.w. mix of the 2 elements
            else:
                # Example search: 'Pd0.4Pt0.6' -> '0.6'
                x = float(re.search(r'(0.\d*)$', frac).group(0))
            # Add color
            new_color = [(x * right_color[i]) + ((1-x) * left_color[i]) for i in range(3)]
            colors.append(new_color)
            # Add point
            xs.append(x)
            ys.append(y)

        # Create scatter
        points_scatter = go.Scatter(
        x=np.array(xs),
        y=np.array(ys),
        mode='markers',
        name='enthalpy',
        hoverinfo='text',
        text=["""<a href="{}/">{}</a>""".format(urls[i], c_name[i]) for i in range(len(urls))],
        hoverlabel=dict(bgcolor='black'),
        marker=dict(
            color=['rgb({}, {}, {})'.format(c[0], c[1], c[2]) for c in colors],
            line=dict(
                color='rgb(0, 0, 0)',
                width=1
                )
            ),
        )

        return points_scatter

    def _create_figure(self, hull_scatter, points_scatter, em1, em2):
        data = [hull_scatter, points_scatter]
        layout = go.Layout(showlegend=False,
                           hovermode='closest',
                           xaxis=dict(
                                title='{}-{}'.format(em1, em2),
                                autorange=True,
                                showgrid=True,
                                zeroline=False,
                                showline=True,
                                showticklabels=True,
                                titlefont=dict(
                                    size=18,
                                )
                            ),
                            yaxis=dict(
                                title='eV/Atom',
                                autorange=True,
                                showgrid=True,
                                zeroline=False,
                                showline=True,
                                showticklabels=True,
                                titlefont=dict(
                                    size=18,
                                )
                            ))
        fig = go.Figure(data=data, layout=layout)
        return fig


# For plotting 3 species alloys (Ternary)
class _Three_Species_Plotter:
    def plot_CHull(self, hull, em1, em2, em3):
        self.em1, self.em2, self.em3 = em1, em2, em3
        # ----- Create scatter for edges -----
        # Grab compounds that make up points for edges
        compounds = [dic['vertices_position'] for dic in hull['facets_data']['3-nary:{}-{}-{}'.format(em1, em2, em3)]]
        # Create scatter for edges
        hull_scatters = self._create_edges_scatter(compounds)

        # ----- Create scatter for points -----
        points_scatter = self._create_points_scatter(hull['points_data'])

        # ----- Plot -----
        fig = self._create_figure(hull_scatters, points_scatter)
        iplot(fig)

    def _create_edges_scatter(self, compounds):
        hull_scatters = []
        points_a, points_b, points_c = [], [], []
        for vertices in compounds:
            point1, point2, point3 = vertices[0], vertices[1], vertices[2]
            point1[2] = 1.0 - (point1[0] + point1[1])
            point2[2] = 1.0 - (point2[0] + point2[1])
            point3[2] = 1.0 - (point3[0] + point3[1])
            # Important note: As and Bs are switched in the json compared to ternary plot points in Plotly
            hull_scatter = {
                'type': 'scatterternary',
                'mode': 'lines',
                'name': 'hull',
                'hoverinfo': 'none',
                'marker': dict(color='rgba(150, 150, 150, 0.4)', size=0.5),
                'a': np.array([point1[1], point2[1], point3[1]]),
                'b': np.array([point1[0], point2[0], point3[0]]),
                'c': np.array([point1[2], point2[2], point3[2]])
            }
            hull_scatters.append(hull_scatter)

        return hull_scatters

    def _create_points_scatter(self, points_data):
        colors, auids, urls, c_name, As, Bs, Cs = [], [], [], [], [], [], []

        # First grab points from the points data
        for compound in points_data:
            # Add AUID and url
            urls.append(compound['url_entry_page'])
            auids.append(compound['auid'])
            # Add compound name
            c_name.append(compound['compound'])
            comp = compound['fractional_compound']
            if compound['nspecies'] == 1:
                if comp == self.em1: vals = [1, 0, 0]
                elif comp ==  self.em2: vals = [0, 1, 0]
                else: vals = [0, 0, 1]
            else:
                em_se = [re.search('{}(0.\d*)'.format(em), comp) for em in [self.em1, self.em2, self.em3]]
                vals = [float(v.group(1)) if v else 0.0 for v in em_se]
            As.append(vals[1])
            Bs.append(vals[0])
            Cs.append(vals[2])
            # Add color
            r, g, b = [1,0,0], [0,1,0], [0,0,1]
            new_color = [(As[-1] * b[i]) + (Bs[-1] * r[i]) + (Cs[-1] * g[i]) for i in range(3)]
            colors.append(new_color)

        points_scatter = {
                'type': 'scatterternary',
                'mode': 'markers',
                'name': 'points',
                'hoverinfo': 'text',
                'a': np.array(As),
                'b': np.array(Bs),
                'c': np.array(Cs),
                'text': ["""<a href="{}/">{}</a>""".format(urls[i], c_name[i]) for i in range(len(urls))],
                'hoverlabel': dict(bgcolor='black'),
                'marker': dict(
                size = 7,
                color=['rgb({}, {}, {})'.format(c[0], c[1], c[2]) for c in colors],
                line=dict(
                    color='rgb(0, 0, 0)',
                    width=1
                    )
                )
            }

        return points_scatter

    def _create_figure(self, hull_scatters, points_scatter):
        data = hull_scatters
        data.append(points_scatter)

        layout = {
            'ternary':{
                'sum': 1.0,
                'aaxis': dict(title=self.em2, showgrid=False, showticklabels=False, titlefont=dict(color='blue')),
                'baxis': dict(title=self.em1, showgrid=False, showticklabels=False, titlefont=dict(color='red')),
                'caxis': dict(title=self.em3, showgrid=False, showticklabels=False, titlefont=dict(color='green'))
            },
            'showlegend': False,
            'hovermode': 'closest',
        }

        fig = {'data': data, 'layout': layout}
        return fig

## Set <i>aflow_path</i>
If your aflow is not installed in home then replace <i>aflow_path</i> below with the path

In [ ]:
aflow_path = '~/bin/aflow'

#initialize classes
chull = CHull(aflow_executable = aflow_path)
chull_plotter = CHullPlotter(aflow_executable = aflow_path)

## Example binary hull (PdPt)
Double click to the right of point to go to compound link
<br>
Be patient when you see the * to the left

### <i>get_hull</i>

In [ ]:
alloy = 'PdPt'
hull_data = chull.get_hull(alloy = alloy, options = '--keep=log')
pprint(hull_data)

### <i>get_distance_to_hull</i> from alloy

In [ ]:
output = chull.get_distance_to_hull(alloy = alloy, off_hull_point = 'aflow:71bc1b15525ffa35', options = '--keep=log')
pprint(output)

#### check files created (<code>aflow_PdPt_hull.log</code>)

In [ ]:
! ls

In [ ]:
! cat aflow_PdPt_hull.log

### <i>get_distance_to_hull</i> from pre-calculated hull

In [ ]:
output = chull.get_distance_to_hull(hull = hull_data, off_hull_point = 'aflow:71bc1b15525ffa35')
pprint(output)

### <i>get_stability_criterion</i>  from alloy

In [ ]:
output = chull.get_stability_criterion(alloy = alloy, hull_point = 'aflow:f31b0e27897cd162', options = None)
pprint(output)

### <i>get_stability_criterion</i> from pre-calculated hull

In [ ]:
output = chull.get_stability_criterion(hull = hull_data, hull_point = 'aflow:f31b0e27897cd162')
pprint(output)

### <i>get_hull_energy</i>

In [ ]:
output = chull.get_hull_energy(alloy = alloy, composition = [0.5], options = None)
pprint(output)

### <i>plot</i> from alloy

In [ ]:
chull_plotter.plot(alloy = "PdPt")

### <i>plot</i> from pre-calculated hull

In [ ]:
chull_plotter.plot(hull = hull_data)

## Example ternary hull (MnPdPt)
For 3D plot, double click on point and then click on the pop-up banner to go to compound link
<br>
Be patient when you see the * to the left

### <i>plot</i>

In [ ]:
chull_plotter.plot(alloy = 'MnPdPt')

## AFLOW-CHULL README


LATEST VERSION OF THE FILE: <a href="http://materials.duke.edu/AFLOW/README_AFLOW_CHULL.TXT">README</a>

README written by: Corey Oses (<a href="mailto:corey.oses@duke.edu">corey.oses@duke.edu</a>)

Citation info: C. Oses, E. Gossett, D. Hicks, F. Rose, E. Perim, I. Takeuchi, S. Sanvito, O. Levy, C. Toher, and S. Curtarolo, <i>AFLOW-CHULL: Cloud-based platform for autonomous phase stability analysis</i>, J. Chem. Inf. Model. <b>58</b>(12), 2477-2490 (2018). 


### GENERAL OVERVIEW:

AFLOW-CHULL is an automated thermodynamic stability analysis tool. Compound data is downloaded from the
AFLOW REST-API/AFLUX Search-API and the minimum energy surface is calculated using a modified
variant (dimensionally-iterative) of the qhull convex hull algorithm:
ACM Trans. on Mathematical Software, <b>22</b>(4):469-483, Dec 1996.
Various thermodynamic properties are extracted, including off-hull (full decomposition reaction,
distances to the hull) and on-hull (equilibrium phases, equivalent ground state structures) features.
Determination of the stability criterion is also available, which has served as a powerful
screening tool for the discovery of new permanent magnets: Science Adv. <b>3</b>(4), e1602241 (2017).

Other important analyses running under the hood include:
 - structure comparison analysis via Burzlaff criteria (<code>--readme=compare</code>)
 - outlier detection and removal (interquartile range).

Various output formats are provided, including plain text, json, PDF, and Jupyter.


### LIST OF AFLOW-CHULL COMMANDS:

<code>aflow --convex_hull=|--chull --alloy=MnPdPt[,AlCuZn,...] [chull_options] [--destination=[DIRECTORY]]</code> : Queries the AFLOW API for relevant entries (see <code>--load_library</code>), calculates the convex hull, and returns the information as a PDF (default, see <code>--output</code>).

<code>--chull</code> : Necessary flag for entering mode for calculating convex hull.

<code>--alloy</code> : Necessary argument, specifies system. This code is not dimension specific, i.e., you can calculate the convex hull for any n-ary system. There are two input modes:  raw (comma-separated) and combinatorial (colon- and comma-separated). Raw input:  <code>--alloy=MnPdPt,AlCuZn</code>. Combinatorial input:  <code>--alloy=Ag,Au:Mn</code>.  This is interpreted as <code>--alloy=AgMn,AuMn</code>.
            
<code>[--destination=[DIRECTORY]]</code> : Optional argument, specify the directory for the output. Default is <code>--destination=./</code>.

#### chull options

<code>[--usage]</code> : Returns usage commands and options.

<code>[--print=|--p=|--output=|--o=latex|pdf|json|text|jupyter|jupyter2|jupyter3]</code> : Select the output format. <code>--print=latex|pdf</code> are the same (.pdf). <code>--print=json|text</code> have the following extensions:  .json and .txt. <code>--print=jupyter|jupyter2|jupyter3</code> creates a jupter notebook json file that plots a convex hull for the specified alloy (jupyter3 is default). Default is <code>--print=pdf</code>.

<code>[--screen_only]</code> : Output is direct to screen (stdout) instead of writing to a file. All logging output is surpressed.

<code>[--keep=log|--keep_log|--keeplog|--log]</code> : Prints a log file of relevant output.

<code>[--keep=tex,log]</code> : Keep both.

##### LOADING OPTIONS
  
<code>[--load_library=|--loadlibrary=|--ll=icsd|lib1|lib2|lib3]</code> : Specify libraries from which to load. Default: <code>--load_library=icsd,lib2,lib3</code>.

<code>[--load_API|--load_api|--loadapi|--lapi|--api]</code> : Force loading entries from the API (default unless on nietzsche, aflowlib, or habana).

<code> [--load_entries_entry_output|--loadentriesentryoutput|--leo]</code> : Get full output for all entries loaded from the AFLOW API.

<code>[--neglect=|--ban=aflow:bb0d45ab555bc208,aflow:fb9eaa58604ce774]</code> : Ban specific entries from the convex hull calculation, done by AUID.

<code>[--see_neglect|--seeneglect| --sn]</code> : Show why entries were neglected.

<code>[--remove_extreme_points=|--removeextremepoints=|--remove_extrema=|--removeextrema=|--rep=-1000]</code> : Exclude points based on Hf/Ts (floor/ceiling). Units are meV/atom / K.

<code>[--entropic_temperature|--entropictemperature|--entroptemp]</code> : Calculate the Ts convex hull (upper-half). Default is Hf hull (lower-half).

<code>[--include_paw_gga|--paw_gga]</code> : Include calculations run with PAW_GGA. Easily coupled with default PAW_PBE (same level of theory, differently parametrized).

##### ANALYSIS OPTIONS:
        
<code>[--distance_to_hull=|--distancetohull=|--distance2hull=|--dist2hull=|--d2h=aflow:bb0d45ab555bc208,aflow:fb9eaa58604ce774]</code> : Calculates a structure's distance below/above (Hf/Ts) the hull.

<code>[--stability_criterion=|--stabilitycriterion=|--stable_criterion=|--scriterion=|--sc=aflow:bb0d45ab555bc208,aflow:fb9eaa58604ce774]</code> : Calculates the stability criterion of the ground state structure. Will return a warning if structure is not a ground state one. It removes the point from the hull, calculates the new hull, and calculates the distance of this point from below/above (Hf/Ts) hull.

<code>[--hull_formation_enthalpy=|--hull_energy=0.25,0.25]</code> : Returns the value of the convex hull surface at the specified coordinate/concentration. For stoichiometric hulls, provide the reduced composition form, i.e., for the compound Mn<sub>2</sub>PdPt, use <code>--hull_formation_enthalpy=0.5,0.25</code>, where the composition of the last component is implicitly 1-&sum;(0.5+0.25). Providing an additional component (e.g., <code>--dist2hull=0.5,0.25,0.1</code>) results in a rigid shift in the energy axis and, thus, a constant added to the final distance calculated (0.1 is in units of eV for formation energy hulls).

<code>[--skip_structure_comparison|--skipstructruecomparison|--skipstructcomp|--ssc]</code> : structure comparison analysis for determination of equivalent ground state structures is skipped (can be time consuming).

<code>[--skip_stability_criterion_analysis|--skipstabilitycriterionanalysis|--skipscriterion|--sscriterion]</code> : analysis of stability criterion for ground state structures is skipped (can be time consuming).

<code>[--include_skewed_hulls|--include_skewed|--ish]</code> : process hull despite skewed endpoints (|ground state endpoint energy| < 15 meV).

<code>[--include_unreliable_hulls|--include_unreliable|--iuh]</code> : process hull despite poor statistics (binary hull count < 200).

<code>[--include_outliers|--io]</code> : include any detected outliers (output/warning is still shown).

<code>[--force]</code> : force output despite warnings.

##### PDF/LATEX OPTIONS:
  
<code>[--image_only|--imageonly|--image]</code> : Latex/PDF output mode only. Similar to <code>--document_only</code>, but the image dimensions are not necessarily for a standard page. Preferred option for importing into papers/presentations.

<code>[--no_document|--nodocument|--no_doc|--nodoc|--full_page_image|--fullpageimage]</code> : Latex/PDF output mode only. Generates a PDF of just the convex hull illustration (2/3D). Dimensions: 8.5x11 inches (standard page), landscape.

<code>[--document_only|--documentonly|--doc_only|--doconly|--doc]</code> : Latex/PDF output mode only. Exclude convex hull illustration. This is the default for quaternary systems and above.

<code>[--keep=tex|--keep_tex|--keeptex|--tex]</code> : Latex/PDF output mode only. Will keep latex .tex and put it in <code>--destination=[DIRECTORY]</code>.

<code>[--latex_output|--latexoutput]</code> : Latex/PDF output mode only. See full latex output. Good for troubleshooting.

<code>[--latex_interactive|--latexinteractive]</code> : Latex/PDF output mode only. Allows you to interact with latex as it compiles. Good for troubleshooting.

<code>[--light_contrast|--lightcontrast|--lc]</code> : Latex/PDF output mode only. Modifies the convex hull illustration color scheme to be lighter.

<code>[--large_font|--largefont|--large|--lf]</code> : Latex/PDF output mode only. Prints a "larger" font size for convex hull illustration.

<code>[--png_resolution=|--pngresolution=|--pngr=300]</code> : Set PNG resolution (<code>-density</code> in convert).

<code>[--plot_iso_max_latent_heat|--iso_max|--isomax]</code> : Plot iso-max-latent-heat lines for ground state structures. See <a href="http://doi.org/10.1063/1.4902865">doi:10.1063/1.4902865</a>.

For additional information contact: Corey Oses (<a href="mailto:corey.oses@duke.edu">corey.oses@duke.edu</a>)

## Exercises

1. What is the distance of Al<sub>17</sub>Co<sub>12</sub> from the hull?

In [ ]:
#Enter code here

2. What is the stability criterion of Te<sub>2</sub>Zr?

In [ ]:
#Enter code here

3. What is the energy of the CuZr hull at <i>x</i> = 0.5?

In [ ]:
#Enter code here

4. What is the most stable structure on the MnPdPt hull?

In [ ]:
#Enter code here